# Naive Bayesian Classifier
### Q1. Bayes Rule을 이해하고 Naive  Bayes classifier가 사용하는 사후 확률 계산 과정을 서술하세요.

- Bayes Rule:   
$P(w_i|x) = \frac{P(x|w_i)|P(w_i)}{P(x)} = \frac{P(x|w_i) P(w_i)}{\Sigma_j P(x|w_j)P(w_j)}$
  -
  - $P(x|w_i)\text{: 사후 확률, posterior}\\
P(x|w_i) \text{: 가능도/우도, likelihood}\\
P(w_i) \text{: 사전 확률, prior}\\
P(x) \text{: 증거, evidence}$

A1. 사전 확률(Prior Probability) 을 구하고 우도를 구하고 이를 이용해 계산합니다

### Q2. Naive Bayes Classification 방법을 이용해서 다음 생성된 리뷰 데이터에 기반한 감정 분석을 해봅시다.

In [1]:
# pip install pandas
import pandas as pd
import re

In [2]:
# 리뷰 데이터 생성
data = {
    'review': [
        'I love this great product! It exceeded my expectations.',
        'The Worst purchase I have ever made. Completely useless.',
        'It is an average product, nothing special but not terrible either.',
        'Great service and who can help but love this design? Highly recommend!',
        'Terrible experience, I will never buy from this poor brand again.',
        'It’s acceptable, but I expected better service, not just an acceptable one.',
        'Absolutely wonderful! I am very satisfied with this great service.',
        'The quality is poor and it broke after one use. Terrible enough!',
        'Acceptable product for the price, but there are better options out there.',
        'Great quality and fast shipping with wonderful service! I love it'
    ],
    'sentiment': [
        'positive', 'negative', 'neutral', 'positive', 'negative',
        'neutral', 'positive', 'negative', 'neutral', 'positive',
    ]
}
df = pd.DataFrame(data)
df.head()

,review,sentiment
0,I love this great product! It exceeded my expe...,positive
1,The Worst purchase I have ever made. Completel...,negative
2,"It is an average product, nothing special but ...",neutral
3,Great service and who can help but love this d...,positive
4,"Terrible experience, I will never buy from thi...",negative


In [3]:
# 불용어 리스트 정의
stopwords = ['i', 'my', 'am', 'this', 'it', 'its', 'an', 'a', 'the', 'is', 'are', 'and', 'product', 'service']

In [4]:
# 텍스트 전처리 함수 정의
def preprocess_text(text):
    # 소문자로 변환
    text = text.lower()
    # 특수 기호 제거
    text = re.sub(r'[^a-z\s]', '', text)
    # 불용어 제거
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords]
    return ' '.join(filtered_words)

# 모든 리뷰에 대해 전처리 수행
df['review'] = df['review'].apply(preprocess_text)

기본적인 데이터 전처리가 완료되었습니다!
이제부터 직접 나이브 베이지안 분류를 수행해 봅시다.  
우리가 분류하고자 하는 문장은 총 두가지 입니다.  
전처리가 완료되었다고 치고,   
첫번째 문장은 **'love, great, awesome'**,  
두번째 문장은 **'terrible, not, never'** 입니다. 

사전 확률 $P(positive), P(negative), P(neutral)$을 구합니다. 

In [12]:
df

,review,sentiment
0,love great exceeded expectations,positive
1,worst purchase have ever made completely useless,negative
2,average nothing special but not terrible either,neutral
3,great who can help but love design highly reco...,positive
4,terrible experience will never buy from poor b...,negative
5,acceptable but expected better not just accept...,neutral
6,absolutely wonderful very satisfied with great,positive
7,quality poor broke after one use terrible enough,negative
8,acceptable for price but there better options ...,neutral
9,great quality fast shipping with wonderful love,positive


In [15]:
P_positive = len(df[df['sentiment'] == 'positive']) / len(df)
P_negative = len(df[df['sentiment'] == 'negative']) / len(df)
P_neutral = len(df[df['sentiment'] == 'neutral']) / len(df)

print("P(positive):", P_positive)
print("P(negative):", P_negative)
print("P(neutral):", P_neutral)

P(positive): 0.4
P(negative): 0.3
P(neutral): 0.3


가능도를 구하기 위한 확률들을 계산합니다.  
예: 첫번째 문장 분류를 위해서는, $P(love|positive), P(great|positive), P(awesome|positive)\\
P(love|negative), P(great|negative), P(awesome|negative)\\
P(love|neutral), P(great|neutral), P(great|neutral)$를 구합니다.

이 때 CountVectorizer를 사용하여 도출한 단어 벡터를 활용하면 확률들을 간편하게 구할 수 있습니다.  
참고: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
review_array = vectorizer.fit_transform(df['review']).toarray()
review_array

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 2, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 

In [8]:
vectorizer.get_feature_names_out()

array(['absolutely', 'acceptable', 'after', 'again', 'average', 'better',
       'brand', 'broke', 'but', 'buy', 'can', 'completely', 'design',
       'either', 'enough', 'ever', 'exceeded', 'expectations', 'expected',
       'experience', 'fast', 'for', 'from', 'great', 'have', 'help',
       'highly', 'just', 'love', 'made', 'never', 'not', 'nothing', 'one',
       'options', 'out', 'poor', 'price', 'purchase', 'quality',
       'recommend', 'satisfied', 'shipping', 'special', 'terrible',
       'there', 'use', 'useless', 'very', 'who', 'will', 'with',
       'wonderful', 'worst'], dtype=object)

In [9]:
vectorizer.vocabulary_

{'love': 28,
 'great': 23,
 'exceeded': 16,
 'expectations': 17,
 'worst': 53,
 'purchase': 38,
 'have': 24,
 'ever': 15,
 'made': 29,
 'completely': 11,
 'useless': 47,
 'average': 4,
 'nothing': 32,
 'special': 43,
 'but': 8,
 'not': 31,
 'terrible': 44,
 'either': 13,
 'who': 49,
 'can': 10,
 'help': 25,
 'design': 12,
 'highly': 26,
 'recommend': 40,
 'experience': 19,
 'will': 50,
 'never': 30,
 'buy': 9,
 'from': 22,
 'poor': 36,
 'brand': 6,
 'again': 3,
 'acceptable': 1,
 'expected': 18,
 'better': 5,
 'just': 27,
 'one': 33,
 'absolutely': 0,
 'wonderful': 52,
 'very': 48,
 'satisfied': 41,
 'with': 51,
 'quality': 39,
 'broke': 7,
 'after': 2,
 'use': 46,
 'enough': 14,
 'for': 21,
 'price': 37,
 'there': 45,
 'options': 34,
 'out': 35,
 'fast': 20,
 'shipping': 42}

In [10]:
frequency_matrix = pd.DataFrame(review_array, columns = vectorizer.get_feature_names_out())
frequency_matrix = pd.concat([df['sentiment'], frequency_matrix], axis=1)
frequency_matrix

,sentiment,absolutely,acceptable,after,again,average,better,brand,broke,but,...,terrible,there,use,useless,very,who,will,with,wonderful,worst
0,positive,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,negative,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,neutral,0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
3,positive,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,negative,0,0,0,1,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,0
5,neutral,0,2,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
6,positive,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,1,0
7,negative,0,0,1,0,0,0,0,1,0,...,1,0,1,0,0,0,0,0,0,0
8,neutral,0,1,0,0,0,1,0,0,1,...,0,2,0,0,0,0,0,0,0,0
9,positive,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0


In [56]:
# 위와 같이 조건부 확률을 구하는 코드를 작성해주세요

# 𝑃(𝑙𝑜𝑣𝑒|𝑝𝑜𝑠𝑖𝑡𝑖𝑣𝑒),𝑃(𝑔𝑟𝑒𝑎𝑡|𝑝𝑜𝑠𝑖𝑡𝑖𝑣𝑒),𝑃(𝑎𝑤𝑒𝑠𝑜𝑚𝑒|𝑝𝑜𝑠𝑖𝑡𝑖𝑣𝑒)
# 𝑃(𝑙𝑜𝑣𝑒|𝑛𝑒𝑔𝑎𝑡𝑖𝑣𝑒),𝑃(𝑔𝑟𝑒𝑎𝑡|𝑛𝑒𝑔𝑎𝑡𝑖𝑣𝑒),𝑃(𝑎𝑤𝑒𝑠𝑜𝑚𝑒|𝑛𝑒𝑔𝑎𝑡𝑖𝑣𝑒)
# 𝑃(𝑙𝑜𝑣𝑒|𝑛𝑒𝑢𝑡𝑟𝑎𝑙),𝑃(𝑔𝑟𝑒𝑎𝑡|𝑛𝑒𝑢𝑡𝑟𝑎𝑙),𝑃(𝑔𝑟𝑒𝑎𝑡|𝑛𝑒𝑢𝑡𝑟𝑎𝑙)
cnt_pos = len(frequency_matrix[frequency_matrix['sentiment'] == 'positive'])
cnt_nega = len(frequency_matrix[frequency_matrix['sentiment'] == 'negative'])
cnt_neu = len(frequency_matrix[frequency_matrix['sentiment'] == 'neutral'])

sentiment = ['positive', 'negative', 'neutral']
review1=['love', 'great', 'awesome']
review2 = ['terrible', 'not', 'never']

post1 = []
post2 = []
for i in review1:    
    p = []
    for j in sentiment:
        if i not in frequency_matrix.columns:
            print("P("+i+"/"+j+"): 0")
            p.append(0)
        else: 
            a= len(frequency_matrix[(frequency_matrix['sentiment'] == j) & (frequency_matrix[i]!=0)])
            b= len(frequency_matrix[(frequency_matrix['sentiment'] == j)])
            print("P("+i+"/"+j+")", a/b )    
            p.append(a/b)
    post1.append(p)
                  
                
print("\n")

for i in review2:
    p = []
    for j in sentiment:
        if i not in frequency_matrix.columns:
            print("P("+i+"/"+j+"): 0")
            p.append(0)
        else: 
            a= len(frequency_matrix[(frequency_matrix['sentiment'] == j) & (frequency_matrix[i]!=0)])
            b= len(frequency_matrix[(frequency_matrix['sentiment'] == j)])
            print("P("+i+"/"+j+")", a/b )    
            p.append(a/b)
    post2.append(p)

P(love/positive) 0.75
P(love/negative) 0.0
P(love/neutral) 0.0
P(great/positive) 1.0
P(great/negative) 0.0
P(great/neutral) 0.0
P(awesome/positive): 0
P(awesome/negative): 0
P(awesome/neutral): 0


P(terrible/positive) 0.0
P(terrible/negative) 0.6666666666666666
P(terrible/neutral) 0.3333333333333333
P(not/positive) 0.0
P(not/negative) 0.0
P(not/neutral) 0.6666666666666666
P(never/positive) 0.0
P(never/negative) 0.3333333333333333
P(never/neutral) 0.0


독립성 가정을 이용하여 가능도(likelihood)를 구합니다.  
첫번째 문장 예시: $P(love, great, awesome|positive), P(love, great, awesome|negative), P(love, great, awesome|neutral)$

In [66]:
# positive_df = frequency_matrix[frequency_matrix['sentiment'] == 'negative']

# # 각 열에서 값이 0이 아닌 열의 개수 계산
# non_zero_counts = (positive_df.iloc[:, 1:] != 0).sum()

# # 값이 0이 아닌 열의 개수
# non_zero_columns_count = (non_zero_counts > 0).sum()

# print("Sentiment가 'positive'이고 값이 0이 아닌 열의 개수:", non_zero_columns_count)

In [61]:
awesome_pos= 1/(19+ len(vectorizer.vocabulary_))
post1[2][0] = awesome_pos* 1*0.75

In [68]:
# 가능도 구하는 코드를 작성해주세요.

result1 = []
result2 = []
# 배열의 길이만큼 반복하며 인덱스끼리 곱셈 수행
for i in range(len(post1[0])):
    product = 1
    for row in post1:
        product *= row[i]
    result1.append(product)

for i in range(len(post2[0])):
    product = 1
    for row in post2:
        product *= row[i]
    result2.append(product)

print("𝑃(𝑙𝑜𝑣𝑒,𝑔𝑟𝑒𝑎𝑡,𝑎𝑤𝑒𝑠𝑜𝑚𝑒|𝑝𝑜𝑠𝑖𝑡𝑖𝑣𝑒)", result1[0])
print("𝑃(𝑙𝑜𝑣𝑒,𝑔𝑟𝑒𝑎𝑡,𝑎𝑤𝑒𝑠𝑜𝑚𝑒|𝑛𝑒𝑔𝑎𝑡𝑖𝑣𝑒)", result1[1])
print("𝑃(𝑙𝑜𝑣𝑒,𝑔𝑟𝑒𝑎𝑡,𝑎𝑤𝑒𝑠𝑜𝑚𝑒|𝑛𝑒𝑢𝑡𝑟𝑎𝑙)", result1[2])


print("𝑃(terrible, not, never|𝑝𝑜𝑠𝑖𝑡𝑖𝑣𝑒)", result2[0])
print("𝑃(terrible, not, never|𝑛𝑒𝑔𝑎𝑡𝑖𝑣𝑒)", result2[1])
print("𝑃(terrible, not, never|𝑛𝑒𝑢𝑡𝑟𝑎𝑙)", result2[2])







𝑃(𝑙𝑜𝑣𝑒,𝑔𝑟𝑒𝑎𝑡,𝑎𝑤𝑒𝑠𝑜𝑚𝑒|𝑝𝑜𝑠𝑖𝑡𝑖𝑣𝑒) 0.007705479452054794
𝑃(𝑙𝑜𝑣𝑒,𝑔𝑟𝑒𝑎𝑡,𝑎𝑤𝑒𝑠𝑜𝑚𝑒|𝑛𝑒𝑔𝑎𝑡𝑖𝑣𝑒) 0.0
𝑃(𝑙𝑜𝑣𝑒,𝑔𝑟𝑒𝑎𝑡,𝑎𝑤𝑒𝑠𝑜𝑚𝑒|𝑛𝑒𝑢𝑡𝑟𝑎𝑙) 0.0
𝑃(terrible, not, never|𝑝𝑜𝑠𝑖𝑡𝑖𝑣𝑒) 0.0
𝑃(terrible, not, never|𝑛𝑒𝑔𝑎𝑡𝑖𝑣𝑒) 0.0
𝑃(terrible, not, never|𝑛𝑒𝑢𝑡𝑟𝑎𝑙) 0.0


위에서 구한 사전 확률과 가능도를 이용하여 타겟 문장이 positive, negative, neutral일 확률을 구하고 최종적으로 어떤 감성일지 분석해봅니다.

In [74]:
len(frequency_matrix[(frequency_matrix['love']!=0) |(frequency_matrix['great']!=0) ])

4

In [76]:
import numpy as np
# 최종 확률 구하는 코드를 작성해주세요.
# 첫번째 문장

print("P(positive|target_review1): ",P_positive* result1[0]/(len(frequency_matrix[(frequency_matrix['love']!=0) |(frequency_matrix['great']!=0) ])/10))

# P(negative|target_review1)

print("P(negative|target_review1): ",P_negative* result1[1]/(len(frequency_matrix[(frequency_matrix['love']!=0) |(frequency_matrix['great']!=0) ])/10))
# P(neutral|target_review1)
print("P(neutral|target_review1): ",P_neutral* result1[2]/(len(frequency_matrix[(frequency_matrix['love']!=0) |(frequency_matrix['great']!=0) ])/10))


# 두번째 문장
# P(positive|target_review2)
print("P(positive|target_review1): ",P_positive* result2[0]/(len(frequency_matrix[(frequency_matrix['never']!=0)| (frequency_matrix['terrible']!=0) |(frequency_matrix['not']!=0) ])/10))


# P(negative|target_review2)
print("P(negative|target_review1): ",P_negative* result2[1]/(len(frequency_matrix[(frequency_matrix['never']!=0)| (frequency_matrix['terrible']!=0) |(frequency_matrix['not']!=0) ])/10))


# P(neutral|target_review2)
print("P(neutral|target_review1): ",P_neutral* result2[2]/(len(frequency_matrix[(frequency_matrix['never']!=0)| (frequency_matrix['terrible']!=0) |(frequency_matrix['not']!=0) ])/10))




P(positive|target_review1):  0.007705479452054794
P(negative|target_review1):  0.0
P(neutral|target_review1):  0.0
P(positive|target_review1):  0.0
P(negative|target_review1):  0.0
P(neutral|target_review1):  0.0


A2-1.   
Target review1의 분류 결과:   
Target review2의 분류 결과: 부정

Q2-2. 나이브 베이지안 기반 확률을 구하는 과정에서 어떤 문제점을 발견할 수 있었나요? 그리고 그 문제를 해결하기 위한 방법에 대해 간략하게 조사 및 서술해 주세요. (힌트: Laplace smoothing)

A2-2. awesome 같은 없는 단어가 있으면 0이 되어버림. 이를 방지하기위해 Naive Bayes 분류기에서 범주형 데이터의 확률을 계산할 때, 빈도 기반 확률이 0이 되는 것을 방지하기 위해 라플라스 스무딩 사용